In [14]:
import numpy as np
import pandas as pd
import math
import sklearn.preprocessing
import datetime
from TimeBasedCV import TimeBasedCV
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import make_scorer, r2_score
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.cross_decomposition import PLSRegression
import pickle 

pd.set_option('display.max_rows', None)
# more

In [15]:
# df = pd.read_csv('factors_2002.csv', parse_dates=['DATE'])

In [16]:

# with open('data/features_1965.pkl', 'wb') as f:
#     pickle.dump(df, f)

with open('data/features_1965.pkl', 'rb') as f:
    df = pickle.load(f)
    print(df.head())



   permno       DATE       mvel1      beta    betasq     chmom     dolvol  \
0   10401 1970-02-27  26227356.0  0.253755  0.064391 -0.057929  13.348086   
1   10604 1970-02-27   3196008.0  0.743947  0.553457 -0.228256  13.348086   
2   10786 1970-02-27   1133566.5  0.608122  0.369813 -0.046833  12.456023   
3   10890 1970-02-27   2662344.0  0.833271  0.694341  0.001993  13.348086   
4   11260 1970-02-27   1342376.0  1.059627  1.122808  0.034205  13.348086   

    idiovol    indmom     mom1m  ...  macro_ep  macro_bm  macro_ntis  \
0  0.019852 -0.112202 -0.017995  ...  2.757287  0.670148    0.035855   
1  0.044046 -0.252877 -0.164006  ...  2.757287  0.670148    0.035855   
2  0.025765 -0.302840 -0.050691  ...  2.757287  0.670148    0.035855   
3  0.035138 -0.127664 -0.018268  ...  2.757287  0.670148    0.035855   
4  0.032067 -0.349402 -0.185455  ...  2.757287  0.670148    0.035855   

   macro_tbl  macro_tms  macro_dfy  macro_svar  macro_mkt-rf  macro_hml  \
0     0.0713    -0.0126     0

In [17]:
train = df.loc[df["DATE"].dt.year<2014 ]
test = df.loc[df['DATE'].dt.year>=2015]

In [18]:
#Sort observations by date and stock id
train[train.columns[2:]] = train[train.columns[2:]].astype('float32')
train = train.sort_values(by = ['DATE', 'permno'], ascending = True)
train.head()


C:\Users\asealy\AppData\Local\Temp\ipykernel_18808\1940433511.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[train.columns[2:]] = train[train.columns[2:]].astype('float32')


,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,macro_ep,macro_bm,macro_ntis,macro_tbl,macro_tms,macro_dfy,macro_svar,macro_mkt-rf,macro_hml,macro_smb
0,10401,1970-02-27,26227356.0,0.253755,0.064391,-0.057929,13.348085,0.019852,-0.112202,-0.017995,...,2.757288,0.670147,0.035855,0.0713,-0.0126,0.0085,0.001059,5.13,3.93,-2.58
1,10604,1970-02-27,3196008.0,0.743947,0.553457,-0.228256,13.348085,0.044046,-0.252877,-0.164006,...,2.757288,0.670147,0.035855,0.0713,-0.0126,0.0085,0.001059,5.13,3.93,-2.58
2,10786,1970-02-27,1133566.5,0.608122,0.369813,-0.046833,12.456023,0.025765,-0.302840,-0.050691,...,2.757288,0.670147,0.035855,0.0713,-0.0126,0.0085,0.001059,5.13,3.93,-2.58
3,10890,1970-02-27,2662344.0,0.833271,0.694341,0.001993,13.348085,0.035138,-0.127664,-0.018268,...,2.757288,0.670147,0.035855,0.0713,-0.0126,0.0085,0.001059,5.13,3.93,-2.58
4,11260,1970-02-27,1342376.0,1.059626,1.122808,0.034205,13.348085,0.032067,-0.349402,-0.185455,...,2.757288,0.670147,0.035855,0.0713,-0.0126,0.0085,0.001059,5.13,3.93,-2.58


In [19]:
train['permno2'] = train['permno'].copy()
train['DATE2'] = train['DATE'].copy()
train = train.set_index(['DATE2','permno2'])

#Make a copy of  the "me" variable (market equity) before rank standartization to use afterwards for value weighting
train['mvel12'] = train['mvel1'].copy()

In [20]:
p=0.2 
train_large= train.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)  
train_small = train.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x)*p),'mvel1')).reset_index(drop=True)  


C:\Users\asealy\AppData\Local\Temp\ipykernel_18808\1162568465.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_large= train.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)
C:\Users\asealy\AppData\Local\Temp\ipykernel_18808\1162568465.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_small = train.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x

In [21]:
features = train.columns[~train.columns.isin(['DATE', 'DATE2', "mvel2",'sic2' ,'permno',"permno2",'risk_premium'])].tolist()
train[features]=train.groupby('DATE')[features].rank(pct=True)

train[features] = 2*train[features] - 1

features = test.columns[~test.columns.isin(['DATE', 'DATE2', "mvel2",'sic2' ,'permno',"permno2",'risk_premium'])].tolist()
test[features]=test.groupby('DATE')[features].rank(pct=True)

test[features] = 2*test[features] - 1

C:\Users\asealy\AppData\Local\Temp\ipykernel_18808\1884498540.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[features]=test.groupby('DATE')[features].rank(pct=True)
C:\Users\asealy\AppData\Local\Temp\ipykernel_18808\1884498540.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[features] = 2*test[features] - 1


In [ ]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   freq='months')

features = train.columns[~train.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()

X = train[features]
y = train[['risk_premium']]

#Empty containers to save results from each window

#Empty containers to save results from each window
predictions = []
y_val_list = []
dates = []
dic_r2_all = {}


numpc_time = {}

numpc = np.arange(1, 20, 1).tolist()
mse = np.full((len(numpc),1), np.nan, dtype=np.float32)

for train_index, val_index  in tscv.split(X, first_split_date= datetime.date(1986,1,31), second_split_date= datetime.date(1988,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]
    
    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]
    
    for i in range(len(numpc)):
        pls_val = PLSRegression(n_components = numpc[i], scale = False)
        pls_val.fit(X_train, y_train)
        Yval_predict=pls_val.predict(X_val)
        Yval_predict = Yval_predict.ravel()
        mse[i,0] = np.sqrt(mean_squared_error(y_val, Yval_predict))
      
    optim_numpc = numpc[np.argmin(mse)]
    
    pls = PLSRegression(n_components=35, scale = False)
    pls.fit(X_train, y_train)
    
    preds_train = pls.predict(X_train)
    preds_val = pls.predict(X_val)
    predictions.append(preds_val)
    y_val_list.append(y_val)

    dates.append(y_val.index)

    r2_train = 1-np.sum(pow(y_train-preds_train,2))/np.sum(pow(y_train,2))
    r2_val = 1-np.sum(pow(y_val-preds_val,2))/np.sum(pow(y_val,2))
    dic_r2_all["r2." + str(y_val.index)] = r2_val


predictions_all_full = np.concatenate(predictions, axis=0)
y_test_list_all_full = np.concatenate(y_val_list, axis=0)
dates_all_full = np.concatenate(dates, axis=0)

R2FULL = 1-np.sum(pow(y_test_list_all_full-predictions_all_full,2))/np.sum(pow(y_test_list_all_full,2))
print("R2OOS Huber Regression: ", R2FULL)

Train period: 1981-01-31 - 1986-01-31 ,val period: 1986-01-31 - 1988-01-31 # train records 20125 ,# val records 13057
Train period: 1982-01-31 - 1987-01-31 ,val period: 1987-01-31 - 1989-01-31 # train records 22963 ,# val records 12572
Train period: 1983-01-31 - 1988-01-31 ,val period: 1988-01-31 - 1990-01-31 # train records 26711 ,# val records 12781
Train period: 1984-01-31 - 1989-01-31 ,val period: 1989-01-31 - 1991-01-31 # train records 28226 ,# val records 13403
Train period: 1985-01-31 - 1990-01-31 ,val period: 1990-01-31 - 1992-01-31 # train records 30708 ,# val records 13616
Train period: 1986-01-31 - 1991-01-31 ,val period: 1991-01-31 - 1993-01-31 # train records 32391 ,# val records 15806
Train period: 1987-01-31 - 1992-01-31 ,val period: 1992-01-31 - 1994-01-31 # train records 33038 ,# val records 17371
Train period: 1988-01-31 - 1993-01-31 ,val period: 1993-01-31 - 1995-01-31 # train records 35140 ,# val records 18821
Train period: 1989-01-31 - 1994-01-31 ,val period: 1994-

c:\Users\asealy\AppData\Local\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\asealy\AppData\Local\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\asealy\AppData\Local\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)


ValueError: need at least one array to concatenate

In [11]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')

features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()

X = df_large[features]
y = df_large[['risk_premium']]

#Empty containers to save results from each window

#Empty containers to save results from each window
predictions = []
y_train_list = []
y_val_list = []
y_test_list =[]
dates = []
dic_r2_all = {}

numpc_time = {}

numpc = np.arange(1, 20, 1).tolist()
mse = np.full((len(numpc),1), np.nan, dtype=np.float32)



for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(2008,1,31), second_split_date= datetime.date(2010,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]
    
    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]
    
    
    # for i in range(len(numpc)):
    #     pls_val = PLSRegression(n_components = numpc[i], scale = False)
    #     pls_val.fit(X_train, y_train)
    #     Yval_predict=pls_val.predict(X_val)
    #     Yval_predict = Yval_predict.ravel()
    #     mse[i,0] = np.sqrt(mean_squared_error(y_val, Yval_predict))
       
       
      
    # optim_numpc = numpc[np.argmin(mse)]
    
    pls = PLSRegression(n_components=35, scale = False)
    pls.fit(np.concatenate((X_train, X_val)), np.concatenate((y_train, y_val)))

    pls.fit(X_train, y_train)
    
    y_pred_train = pls.predict(X_train)
    y_train_list.append(r2_score(y_train, y_pred_train))
    
    y_pred_val = pls.predict(X_val)
    y_val_list.append(r2_score(y_val, y_pred_val))

    y_pred_test = pls.predict(X_test)
    y_test_list.append(r2_score(y_test, y_pred_test))


r2_val_large = np.mean(y_val_list)
print(r2_val_large)

Train period: 2003-01-31 - 2008-01-31 ,val period: 2008-01-31 - 2010-01-31 , Test period 2010-01-31 - 2011-01-31 # train records 121078 ,# val records 46299 , # test records 18787
Train period: 2004-01-31 - 2009-01-31 ,val period: 2009-01-31 - 2011-01-31 , Test period 2011-01-31 - 2012-01-31 # train records 120082 ,# val records 39763 , # test records 20259
Train period: 2005-01-31 - 2010-01-31 ,val period: 2010-01-31 - 2012-01-31 , Test period 2012-01-31 - 2013-01-31 # train records 119083 ,# val records 39046 , # test records 19940
Train period: 2006-01-31 - 2011-01-31 ,val period: 2011-01-31 - 2013-01-31 , Test period 2013-01-31 - 2014-01-31 # train records 113784 ,# val records 40199 , # test records 19738
Train period: 2007-01-31 - 2012-01-31 ,val period: 2012-01-31 - 2014-01-31 , Test period 2014-01-31 - 2015-01-31 # train records 109858 ,# val records 39678 , # test records 22032
Train period: 2008-01-31 - 2013-01-31 ,val period: 2013-01-31 - 2015-01-31 , Test period 2015-01-31 

In [12]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')

features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()

X = df_small[features]
y = df_small[['risk_premium']]

#Empty containers to save results from each window

#Empty containers to save results from each window
predictions = []
y_train_list = []
y_val_list = []
y_test_list =[]
dates = []
dic_r2_all = {}

numpc_time = {}

numpc = np.arange(1, 20, 1).tolist()
mse = np.full((len(numpc),1), np.nan, dtype=np.float32)



for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(2008,1,31), second_split_date= datetime.date(2010,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]
    
    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]
    
    
    # for i in range(len(numpc)):
    #     pls_val = PLSRegression(n_components = numpc[i], scale = False)
    #     pls_val.fit(X_train, y_train)
    #     Yval_predict=pls_val.predict(X_val)
    #     Yval_predict = Yval_predict.ravel()
    #     mse[i,0] = np.sqrt(mean_squared_error(y_val, Yval_predict))
       
       
      
    # optim_numpc = numpc[np.argmin(mse)]
    
    pls = PLSRegression(n_components=35, scale = False)
    pls.fit(np.concatenate((X_train, X_val)), np.concatenate((y_train, y_val)))

    pls.fit(X_train, y_train)
    
    y_pred_train = pls.predict(X_train)
    y_train_list.append(r2_score(y_train, y_pred_train))
    
    y_pred_val = pls.predict(X_val)
    y_val_list.append(r2_score(y_val, y_pred_val))

    y_pred_test = pls.predict(X_test)
    y_test_list.append(r2_score(y_test, y_pred_test))


r2_val_small = np.mean(y_val_list)
print(r2_val_small)

Train period: 2003-01-31 - 2008-01-31 ,val period: 2008-01-31 - 2010-01-31 , Test period 2010-01-31 - 2011-01-31 # train records 121078 ,# val records 46299 , # test records 18787
Train period: 2004-01-31 - 2009-01-31 ,val period: 2009-01-31 - 2011-01-31 , Test period 2011-01-31 - 2012-01-31 # train records 120082 ,# val records 39763 , # test records 20259
Train period: 2005-01-31 - 2010-01-31 ,val period: 2010-01-31 - 2012-01-31 , Test period 2012-01-31 - 2013-01-31 # train records 119083 ,# val records 39046 , # test records 19940
Train period: 2006-01-31 - 2011-01-31 ,val period: 2011-01-31 - 2013-01-31 , Test period 2013-01-31 - 2014-01-31 # train records 113784 ,# val records 40199 , # test records 19738
Train period: 2007-01-31 - 2012-01-31 ,val period: 2012-01-31 - 2014-01-31 , Test period 2014-01-31 - 2015-01-31 # train records 109858 ,# val records 39678 , # test records 22032
Train period: 2008-01-31 - 2013-01-31 ,val period: 2013-01-31 - 2015-01-31 , Test period 2015-01-31 

In [14]:
chart = np.array([[r2_val_full],
                  [r2_val_large ],
                  [r2_val_small]])

r2_lm = pd.DataFrame(chart, columns=['PLS Regression'],
                     index=['Full Sample', 'Large Firms', 'Small Firms'])

r2_lm

,PLS Regression
Full Sample,0.054138
Large Firms,0.135334
Small Firms,0.014686


In [15]:
r2_lm.to_csv(r'r2_PLSReg_model.csv')